In [1]:
import numpy as np
import os
import pickle
from tensorflow.keras.utils import to_categorical

# Load the dataset from pickle file
file_path = "mcts7500_pool.pickle"
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Dataset not found: {file_path}")

# Load the pickle file
with open(file_path, "rb") as f:
    data = pickle.load(f)

# Ensure data is a dictionary
if not isinstance(data, dict):
    raise ValueError("Pickle file does not contain a dictionary.")

# Extract features (X) and labels (y) based on the new keys
if "board_x" in data and "play_y" in data:
    X = np.array(data["board_x"])  # Use 'board_x' instead of 'X'
    y = np.array(data["play_y"])   # Use 'play_y' instead of 'y'
else:
    raise KeyError("Pickle file does not contain expected keys 'board_x' and 'play_y'.")

# Reshape input (Add channel dimension for CNN)
X = X.reshape(-1, 6, 7, 1).astype("float32")

# Normalize to [-1,1] (assuming values are -1, 0, 1)
X = X / np.max(np.abs(X))

# One-hot encode labels (7 possible moves)
y = to_categorical(y, num_classes=7)

# Print final shape
print(f"✅ Successfully Loaded Data!")
print(f"Final X shape: {X.shape}, Final y shape: {y.shape}")


✅ Successfully Loaded Data!
Final X shape: (265620, 6, 7, 1), Final y shape: (265620, 7)


In [2]:
from sklearn.model_selection import train_test_split

# Split dataset (80% training, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Data Shape: {X_train.shape}, Validation Data Shape: {X_val.shape}")


Training Data Shape: (212496, 6, 7, 1), Validation Data Shape: (53124, 6, 7, 1)


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.regularizers import l2

# Optimized CNN Model
model = Sequential([
    # Conv Block 1
    Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001), input_shape=(6, 7, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    # Conv Block 2
    Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    # Conv Block 3
    Conv2D(256, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2), padding='same'),

    # Flatten & Fully Connected Layers
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),

    # Output Layer
    Dense(7, activation='softmax')
])

# Compile Model (Use Adam with Learning Rate Decay)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Show Model Summary
model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 6, 7, 64)            │             640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 6, 7, 64)            │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 3, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 3, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 3, 4, 128)           │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 2, 2, 256)           │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 2, 2, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 7)                   │             903 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 471,047 (1.80 MB)

 Trainable params: 470,151 (1.79 MB)

 Non-trainable params: 896 (3.50 KB)

In [4]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Reduce learning rate dynamically
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)


In [5]:
from tensorflow.keras.callbacks import EarlyStopping

# Improved Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Train Model
history = model.fit(X_train, y_train,
                    validation_data=(X_val, y_val),
                    epochs=150,
                    batch_size=128,
                    callbacks=[early_stopping, lr_scheduler])


Epoch 1/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 212s 125ms/step - accuracy: 0.3429 - loss: 2.0979 - val_accuracy: 0.4278 - val_loss: 1.6088 - learning_rate: 0.0010
Epoch 2/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 262s 125ms/step - accuracy: 0.4376 - loss: 1.5651 - val_accuracy: 0.4418 - val_loss: 1.5263 - learning_rate: 0.0010
Epoch 3/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 264s 127ms/step - accuracy: 0.4556 - loss: 1.4929 - val_accuracy: 0.4554 - val_loss: 1.4908 - learning_rate: 0.0010
Epoch 4/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 259s 125ms/step - accuracy: 0.4706 - loss: 1.4471 - val_accuracy: 0.4646 - val_loss: 1.4621 - learning_rate: 0.0010
Epoch 5/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 261s 124ms/step - accuracy: 0.4777 - loss: 1.4202 - val_accuracy: 0.4642 - val_loss: 1.4517 - learning_rate: 0.0010
Epoch 6/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 203s 122ms/step - accuracy: 0.4854 - loss: 1.3970 - val_accuracy: 0.4755 - val_loss: 1.4199 - learning_rate: 0.0010
Epoch 7/150
1661/1661 ━━━━━━━━━━━━━━━━━━━━ 266

In [6]:
# Evaluate Model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Final Validation Accuracy: {val_acc:.4f}")

1661/1661 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.5576 - loss: 1.2140
Final Validation Accuracy: 0.5558


In [7]:
# Save model
model.save("cnn_connect4_model.h5")
print("Model saved successfully!")

Model saved successfully!
